In [1]:
# S3 prefix
prefix = 'arn:aws:s3:::sagemaker-us-east-2-117588387775'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd

role = 'arn:aws:iam::117588387775:role/service-role/AmazonSageMaker-ExecutionRole-20190513T204887'

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [3]:
WORK_DIRECTORY = '../src/data/processed'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [4]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/application-sample:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

2019-10-05 22:34:04 Starting - Starting the training job...
2019-10-05 22:34:05 Starting - Launching requested ML instances...
2019-10-05 22:34:58 Starting - Preparing the instances for training......
2019-10-05 22:35:55 Downloading - Downloading input data
2019-10-05 22:35:55 Training - Downloading the training image..Starting the training.
Training complete.

2019-10-05 22:36:27 Uploading - Uploading generated training model
2019-10-05 22:36:27 Completed - Training job completed
Training seconds: 45
Billable seconds: 45


In [5]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

-------------------------------------------------------------------------!

In [6]:
shape=pd.read_csv("../src/data/processed/iris.csv", header=None)
sample = shape.sample(3)
sample

,0,1,2,3,4
29,setosa,4.7,3.2,1.6,0.2
64,versicolor,5.6,2.9,3.6,1.3
60,versicolor,5.0,2.0,3.5,1.0


In [7]:
# drop the label column in the training set
sample.drop(shape.columns[[0]],axis=1,inplace=True)
sample

,1,2,3,4
29,4.7,3.2,1.6,0.2
64,5.6,2.9,3.6,1.3
60,5.0,2.0,3.5,1.0


In [8]:
print(predictor.predict(sample.values).decode('utf-8'))

setosa
versicolor
versicolor

